<a href="https://colab.research.google.com/github/Ankan1998/Autoencoders/blob/main/Denoising_Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision import transforms,datasets
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
import torch
import torch.optim as optim
#from torch.autograd import Variable

#### Parameters
----------
image : ndarray
    Input image data. Will be converted to float.
mode : str
    One of the following strings, selecting the type of noise to add:

    'gauss'     Gaussian-distributed additive noise.
    'poisson'   Poisson-distributed noise generated from the data.
    's&p'       Replaces random pixels with 0 or 1.
    'speckle'   Multiplicative noise using out = image + n*image,where
                n is uniform noise with specified mean & variance.


In [ ]:
"""
from PIL import Image
im=Image.open("/content/mt694087.chuvyon_figure5_hires(en-us,msdn.10).png")

image=np.array(im)
image=image.reshape(300,300,1)
imnp=image[...,:3]

Image.fromarray(np.uint8(noi_img)).convert('RGB')
"""

In [2]:
import numpy as np
import os
import cv2
def noisy(noise_typ,image):
  
    #  Gaussian Noise
    if noise_typ == "gauss":
      row,col,ch= image.shape
      mean = 0
      var = 0.1
      sigma = var**0.5
      gauss = np.random.normal(mean,sigma,(row,col,ch))
      gauss = gauss.reshape(row,col,ch)
      noisy = image + gauss
      return noisy

    # Salt and pepper Noise
    elif noise_typ == "s&p":
      row,col,ch = image.shape
      s_vs_p = 0.5
      amount = 0.004
      out = np.copy(image)
      # Salt mode
      num_salt = np.ceil(amount * image.size * s_vs_p)
      coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
      out[coords] = 1

      # Pepper mode
      num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
      coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
      out[coords] = 0
      return out

    # Poisson Noise
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy

    # Speckle Noise
    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy

In [3]:
FMNIST_train=datasets.FashionMNIST('/files/',train=True,download=True)
FMNIST_test=datasets.FashionMNIST('/files/',train=False,download=True)

Extracting /files/FashionMNIST/raw/train-images-idx3-ubyte.gz to /files/FashionMNIST/raw


Extracting /files/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /files/FashionMNIST/raw


Extracting /files/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /files/FashionMNIST/raw




Extracting /files/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /files/FashionMNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [46]:
class data_prep(Dataset):

  def __init__(self,dir,noise_type,transform=None):
    self.dir=dir
    self.noise_typ=noise_type
    self.transform=transform


  def __getitem__(self,index):
    image=np.array(self.dir[index][0],dtype=np.float64)
    if len(image.shape)==2:
      image=image.reshape(image.shape[0],image.shape[1],1)
    label=torch.tensor(self.dir[index][1])
    noisy_image=torch.tensor(noisy(self.noise_typ,image))
    image=torch.from_numpy(image)
    return image,noisy_image,label
  
  def __len__(self):
    return len(self.dir)



In [49]:
train=data_prep(FMNIST_train,"gauss")
test=data_prep(FMNIST_test,"gauss")

In [51]:
batch_size=64

trainloader=DataLoader(train,batch_size=batch_size,shuffle=True)
testloader=DataLoader(test,batch_size=batch_size,shuffle=True)

In [53]:
i,n_i,l=next(iter(trainloader))

In [56]:
class denoising_model(nn.Module):
  def __init__(self):
    super(denoising_model,self).__init__()
    self.encoder=nn.Sequential(
                  nn.Linear(28*28,256),
                  nn.ReLU(True),
                  nn.Linear(256,128),
                  nn.ReLU(True),
                  nn.Linear(128,64),
                  nn.ReLU(True)
        
                  )
    
    self.decoder=nn.Sequential(
                  nn.Linear(64,128),
                  nn.ReLU(True),
                  nn.Linear(128,256),
                  nn.ReLU(True),
                  nn.Linear(256,28*28),
                  nn.Sigmoid(),
                  )
    
 
  def forward(self,x):
    x=self.encoder(x)
    x=self.decoder(x)
    
    return x

In [57]:
model=denoising_model()

In [58]:

#We check whether cuda is available and choose device accordingly
if torch.cuda.is_available()==True:
  device="cuda:0"
else:
  device ="cpu"

  
model=denoising_model().to(device)

In [59]:
criterion=nn.MSELoss()
optimizer=optim.Adam(model.parameters(),lr=0.01,weight_decay=1e-5)